In [ ]:
# import das libs necessárias
import pandas as pd # trabalhar com dataframes
import numpy as np # realizacao de algumas operacoes com matrizes

#imagens
import cv2 # transformacoes faceis em imagens
from PIL import Image # trabalhar com imagens

# ferramentas
import glob # exploracao de diretorios
from pylab import *
import tqdm
import importlib
import time

# plot 
import matplotlib.pyplot as plt # plotagem
%matplotlib inline

# Machine Learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV # Grid Search para rodar todos os parametros

#importando ferramentas já implementadas anteriormente
import sampling #retorna uma amostra para treino
importlib.reload(sampling)
import PCA_and_Inverse
importlib.reload(PCA_and_Inverse)
import feature_selection
importlib.reload(feature_selection)
import Utilities
importlib.reload(Utilities)

In [ ]:
time.sleep(5*60*60)

## Cria uma amostra das imagens do tamanho definido e carrega em dataframes

In [ ]:
#carregando amostra dos dados
sample_size = 1000
X, Y = sampling.get_sample(sample_size)

## Normaliza os dados

In [ ]:
X = sampling.get_min_max_normalization(data=X)

## Extração de características

- Calcula as Componentes principais para os dados carregados

In [ ]:
pca = PCA_and_Inverse.PCA_and_Inverse()
pca_components = pca.get_PCA(X=X, n_components=64)

## Seleção de características

- Calcula as melhores features a serem utilizadas pelos métodos de:
    - Informação Mútua
    - Algoritmo Genético

#### Informação Mútua

In [ ]:
mic_threshold = 0.05
mic_features_df = feature_selection.get_MIC_features(X=X, Y=Y)
mic_features = list(mic_features_df[mic_features_df[0] > mic_threshold].index)

#### Algoritmo Genético

In [ ]:
ga_features = feature_selection.get_GA_features(X=X, Y=Y)
ga_features

## Gera as combinações de parâmetros para rodar a Random Forest

In [ ]:
rnd_forest_clf = RandomForestClassifier()
parameters = [{"n_estimators":[100, 100000], 
               "criterion":["gini", "entropy"],
               "max_depth":[1000, None],
               "min_samples_split":[2, 100],
               "min_samples_leaf":[1, 50],
               "max_features":[1000, "auto", "log2", None],
               "max_leaf_nodes":[500, None]}]
scoring = ["accuracy", "precision", "recall", "roc_auc"]

if not isinstance(parameters, list):
    parameters = [parameters]

param_set = set()
for dct in parameters:
    param_set = param_set.union(set(dct.keys()))

result_column_list= [f'param_{param}' for param in param_set] + [f'mean_test_{score}' for score in scoring]

#### Roda todas as combinações da Random Forest para todas as features

In [ ]:
grid_search_full = GridSearchCV(rnd_forest_clf, parameters, verbose=5, n_jobs=-1, cv=3, scoring=scoring, refit=False)

In [ ]:
grid_search_full.fit(X, Y)

In [ ]:
full_result = pd.DataFrame(grid_search_full.cv_results_)[result_column_list]
full_result.to_csv(f"Sample_{sample_size}_Full.csv")

#### Roda todas as combinações da Random Forest para as features extraídas pelo PCA

In [ ]:
grid_search_pca = GridSearchCV(rnd_forest_clf, parameters, verbose=5, n_jobs=-1, cv=3, scoring=scoring, refit=False)

In [ ]:
grid_search_pca.fit(pca_components, Y)

In [ ]:
pca_result = pd.DataFrame(grid_search_pca.cv_results_)[result_column_list]
pca_result.to_csv(f"Sample_{sample_size}_PCA.csv")

#### Roda todas as combinações da Random Forest para as features selecionadas pela Informação Mútua (MIC)

In [ ]:
grid_search_mic = GridSearchCV(rnd_forest_clf, parameters, verbose=5, n_jobs=-1, cv=3, scoring=scoring, refit=False)

In [ ]:
grid_search_mic.fit(X[mic_features], Y)

In [ ]:
mic_result = pd.DataFrame(grid_search_mic.cv_results_)[result_column_list]
mic_result.to_csv(f"Sample_{sample_size}_MIC.csv")

#### Roda todas as combinações da Random Forest para as features selecionadas pelo Algoritmo Genético (GA)

In [ ]:
grid_search_ga = GridSearchCV(rnd_forest_clf, parameters, verbose=5, n_jobs=-1, cv=3, scoring=scoring, refit=False)

In [ ]:
grid_search_ga.fit(pca_components, Y)

In [ ]:
ga_result = pd.DataFrame(grid_search_ga.cv_results_)[result_column_list]
ga_result.to_csv(f"Sample_{sample_size}_GA.csv")

#### Salva todas as informações geradas em um Excel

In [ ]:
Utilities.to_excel(dataframe=pd.DataFrame([full_result, pca_result, mic_result, ga_result]),
                   name=f"Sample_{sample_size}",
                   sheet_name="Results")